### Notes
* https://medium.com/@luckylwk/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
* http://scikit-learn.org/stable/modules/manifold.html#t-sne
* http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf
* https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling
* https://www.dataquest.io/blog/build-a-data-science-portfolio/
* https://www.youtube.com/watch?v=NEaUSP4YerM
* http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
* https://www.stat.berkeley.edu/~spector/s133/Clus.html
* https://stats.stackexchange.com/questions/10540/how-to-interpret-mean-of-silhouette-plot
* Visualize clusters with and without tSNE
* Need to understand how to explain the variance of costs within the clusters
* Need to plot silhouette charts

In [ ]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
import seaborn as sns
from nltk import word_tokenize
from collections import Counter
from wordcloud import WordCloud
from string import punctuation
from sklearn.decomposition import PCA
import re
%matplotlib inline

In [ ]:
file = 'http://bit.ly/aqmd-violations-final'
titlev = 'http://bit.ly/aqmd-titlev'
reclaim = 'http://bit.ly/aqmd-reclaim'

In [ ]:
df_all = pd.read_csv(file)
df_titleV = pd.read_csv(titlev)
df_reclaim = pd.read_csv(reclaim)

In [ ]:
df_all.head()

In [ ]:
df_all['facility_type'] = ''

In [ ]:
df_all.head()

In [ ]:
df_titleV.head()

In [ ]:
df_reclaim.head()

In [ ]:
reclaim_list = df_reclaim['Fac_id'].tolist()

In [ ]:
titlev_list = df_titleV['Facility ID'].tolist()

In [ ]:
reclaim_titlev_list = [facility for facility in titlev_list if facility in reclaim_list]

In [ ]:
list_error = []
for index, row in df_all.iterrows():
    try:
        pd.to_numeric(row['facility id'])
    except:
        list_error.append(row)
list_error

In [ ]:
for each in list_error:
    print(each['facility id'])

In [ ]:
for each in list_error:
    print(each['notification'])

In [ ]:
df_all.iloc[475]['facility id']
df_all.set_value(475, 'facility id', '50134')

In [ ]:
df_all.iloc[476]
df_all.set_value(476, 'facility id', '50134')

In [ ]:
df_all.iloc[477]
df_all.set_value(477, 'facility id', '50134')

In [ ]:
df_all.iloc[1284]
df_all.set_value(1284, 'facility id', '2344')

In [ ]:
list_error = []
for index, row in df_all.iterrows():
    try:
        pd.to_numeric(row['facility id'])
    except:
        list_error.append(row)
list_error

In [ ]:
for each in list_error:
    print(each['facility id'])

In [ ]:
for each in list_error:
    print(each['notification'])

In [ ]:
df_all.drop(df_all.index[1695])

In [ ]:
list_error = []
for index, row in df_all.iterrows():
    try:
        pd.to_numeric(row['facility id'])
    except:
        list_error.append(row)
        print(index)

In [ ]:
df_all = df_all.drop(df_all.index[[1695,
2066,
2067,
3138,
3140,
3141]])

In [ ]:
list_error = []
for index, row in df_all.iterrows():
    try:
        pd.to_numeric(row['facility id'])
    except:
        list_error.append(row)
        print(index)

In [ ]:
df_all.head()

In [ ]:
index_row = []
for index, row in df_all.iterrows():
    try:
        if float(row['facility id']) in titlev_list:
            df_all.set_value(index, 'facility_type', 't')
        elif float(row['facility id']) in reclaim_list:
            df_all.set_value(index, 'facility_type', 'r')
        else:
            df_all.set_value(index, 'facility_type', 'n')
    except:
        index_row.append(row)
index_row

In [ ]:
index_row = []
for index, row in df_all.iterrows():
    try:
        if float(row['facility id']) in reclaim_titlev_list:
            df_all.set_value(index, 'facility_type', 'rt')
    except:
        index_row.append(row)
index_row

In [ ]:
df_all.head()

In [ ]:
df_all.shape

In [ ]:
df_rt = df_all[(df_all['facility_type'] == 'rt')]
df_rt.head()

In [ ]:
df_facility_average = df_all.groupby(['facility_type'], as_index=False).mean()
df_facility_average

In [ ]:
punctuation_symbols = []
for each in list(punctuation):
    punctuation_symbols.append((each))

In [ ]:
set(df_all['facility_type'].tolist())

In [ ]:
len(df_all['facility_type'].tolist())

In [ ]:
df_all = df_all.dropna(subset=['violation_data'])
df_all.shape

In [ ]:
def remove_puncuation(d):
    for each in punctuation_symbols:
        d = d.replace(each, '')
    return d

In [ ]:
def remove_digits(x):
    x = ''.join([i for i in x if not i.isdigit()])
    return x

In [ ]:
df_all['violation_data'] = df_all.violation_data.apply(remove_digits)
df_all['violation_data'] = df_all.violation_data.apply(remove_puncuation)
df_all['violation_data'] = df_all['violation_data'].str.lower()

In [ ]:
df_all.head()

In [ ]:
df_all['trim_violation_text'] = ''
for index, row in df_all.iterrows():
    df_all.set_value(index, 'trim_violation_text', re.sub(r' +', ' ', str(row['violation_data'])))
df_all.head()

In [ ]:
df_all['violation_data'].isnull().sum()

In [ ]:
facility_type_ = dict()
type_list = ['rt', 't', 'r', 'n']
for each in type_list:
    text = ' '.join(df_all.loc[df_all['facility_type'] == each, 'trim_violation_text'].values)
    facility_type_[each] = word_tokenize(text)

In [ ]:
none_top100 = Counter(facility_type_['n']).most_common(100)
reclaim_top100 = Counter(facility_type_['r']).most_common(100)
titlev_top100 = Counter(facility_type_['t']).most_common(100)
reclaim_titlev_top100 = Counter(facility_type_['rt']).most_common(100)

In [ ]:
def create_wordcloud(d):
    wordcloud = WordCloud(background_color='white',
                          max_words=75, 
                          max_font_size=40,
                          random_state=42).generate(str(d))
    return wordcloud

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(30, 15))

ax = axes[0, 0]
ax.imshow(create_wordcloud(none_top100), interpolation="bilinear")
ax.axis('off')
ax.set_title("Non-RECLAIM/Title V Top 100 Tokens", fontsize=30)

ax = axes[0, 1]
ax.imshow(create_wordcloud(reclaim_top100))
ax.axis('off')
ax.set_title("RECLAIM Top 100 Tokens", fontsize=30)

ax = axes[1, 0]
ax.imshow(create_wordcloud(titlev_top100))
ax.axis('off')
ax.set_title("Title V Top 100 Tokens", fontsize=30)

ax = axes[1, 1]
ax.imshow(create_wordcloud(reclaim_titlev_top100))
ax.axis('off')
ax.set_title("RECLAIM/Title V Top 100 Tokens", fontsize=30)
plt.show()

In [ ]:
plt.subplot(1, 3, 1)
(df_all['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,1500000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$,1,500,000', fontsize=17)

plt.subplot(1, 3, 2)
(df_all['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,25000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$25,000', fontsize=17)

plt.subplot(1, 3, 3)
(np.log(df_all['Total'] + 1)).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,17])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: log(x + 1)', fontsize=17)
plt.show()

#https://www.quora.com/What-are-the-downsides-when-using-log-x-+-1-as-ones-log-transformation
#http://onbiostatistics.blogspot.com/2012/05/logx1-data-transformation.html

In [ ]:
facility_count = df_all[['Total','facility_type']].groupby(['facility_type'], as_index =False).count()
facility_count

In [ ]:
sns.barplot(facility_count['facility_type'], facility_count['Total'])
plt.grid(True)
plt.ylim(0, 3000, 100)
plt.xlabel('Facility Type', fontsize = 14)
plt.ylabel('Number of Violations', fontsize = 14)
plt.title('Number of Violations By Facility Type', fontsize = 14)
plt.show()

In [ ]:
df_none = df_all[df_all['facility_type'] == 'n'].reset_index(drop=True)
df_rt = df_all[df_all['facility_type'] == 'rt'].reset_index(drop=True)
df_r = df_all[df_all['facility_type'] == 'r'].reset_index(drop=True)
df_t = df_all[df_all['facility_type'] == 't'].reset_index(drop=True)

In [ ]:
plt.subplot(1, 3, 1)
(df_none['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,1500000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$,1,500,000', fontsize=17)

plt.subplot(1, 3, 2)
(df_none['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,25000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$25,000', fontsize=17)

plt.subplot(1, 3, 3)
(np.log(df_none['Total'] + 1)).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,17])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: log(x + 1)', fontsize=17)
plt.show()

In [ ]:
plt.subplot(1, 3, 1)
(df_rt['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,1500000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$,1,500,000', fontsize=17)

plt.subplot(1, 3, 2)
(df_rt['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,25000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$25,000', fontsize=17)

plt.subplot(1, 3, 3)
(np.log(df_rt['Total'] + 1)).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,17])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: log(x + 1)', fontsize=17)
plt.show()

In [ ]:
plt.subplot(1, 3, 1)
(df_r['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,1500000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$,1,500,000', fontsize=17)

plt.subplot(1, 3, 2)
(df_r['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,25000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$25,000', fontsize=17)

plt.subplot(1, 3, 3)
(np.log(df_r['Total'] + 1)).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,17])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: log(x + 1)', fontsize=17)
plt.show()

In [ ]:
plt.subplot(1, 3, 1)
(df_t['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,1500000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$,1,500,000', fontsize=17)

plt.subplot(1, 3, 2)
(df_t['Total']).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,25000])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: $0-$25,000', fontsize=17)

plt.subplot(1, 3, 3)
(np.log(df_t['Total'] + 1)).plot.hist(bins=50, figsize=(20,10), edgecolor='white',range=[0,17])
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Frequency', fontsize=17)
plt.tick_params(labelsize=15)
plt.title('Settlement Distribution :: log(x + 1)', fontsize=17)
plt.show()

In [ ]:
sns.set_style('whitegrid')
data = df_all[df_all['Total'] < 20000]
ax = sns.boxplot(x="Total", y="facility_type", data=data)
plt.xlabel('Settlement Cost', fontsize=17)
plt.ylabel('Facility Type', fontsize=17)
plt.show()

In [ ]:
min_df = 0.01
max_df = 0.90
vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))

In [ ]:
X = df_all['violation_data']
y = df_all['facility_type']

In [ ]:
def determine_length(d):
    d = d.split(' ')
    return len(d)

In [ ]:
df_all['violation_length'] = df_all['trim_violation_text'].apply(determine_length)

In [ ]:
df_all.iloc[1]['trim_violation_text'].split(' ')

In [ ]:
df_all.iloc[1]['violation_length']

In [ ]:
df_all.head()

In [ ]:
df_len_cost = df_all[['Total', 'violation_length']].groupby(['violation_length'], as_index=False).mean()
df_len_cost.head()

In [ ]:
plt.figure(figsize = (10, 10))
sns.jointplot(x=df_len_cost["Total"], y=df_len_cost["violation_length"], kind='kde', color = 'b')
plt.show()

In [ ]:
x = np.log(df_len_cost['violation_length'])
y = np.log(df_len_cost['Total'])
plt.scatter(x, y)
plt.plot(x, y)
plt.ylabel('log(Average Settlement Cost)')
plt.xlabel('log(Length of Violation (number of words))')
plt.title('Average Settlement Cost vs Length of Violation')
plt.show()

In [ ]:
x = (df_len_cost['violation_length'])
y = np.log(df_len_cost['Total'])
plt.scatter(x, y)
plt.plot(x, y)
plt.ylabel('log(Average Settlement Cost)')
plt.xlabel('log(Length of Violation (number of words))')
plt.title('Average Settlement Cost vs Length of Violation')
plt.show()

This is a PCA of the initial dataset using `n_components = 2`

In [ ]:
min_df = 0.01
max_df = 0.90
vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))
X_dtm = vect.fit_transform(X)
pca = PCA(n_components = 2)
pca_results = pca.fit_transform(X_dtm.toarray())
print('Size of Feature Vector is: ', len(X_dtm.toarray()[0]))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print('Total Variance', (pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1]) * 100, '%')

In [ ]:
per_var = np.round(pca.explained_variance_ratio_ * 100, decimals = 1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
plt.bar(x = range(1, len(per_var)+1), height=per_var, tick_label = labels)
plt.xlabel('Principal Component', fontsize = 14)
plt.ylabel('% of Explained Variance', fontsize = 14)
plt.show()

In [ ]:
pca_df = pd.DataFrame(pca_results, columns=['x', 'y'])
pca_df['violation_text'] = df_all['violation_data']
pca_df['facility_type'] = df_all['facility_type']
pca_df['Total_Settlement'] = df_all['Total']

In [ ]:
pca_df.head()

In [ ]:
sns.lmplot( x="x", y="y", data=pca_df, fit_reg=False, hue='facility_type', legend=False)
plt.legend(loc='lower right')
plt.xlabel('PC-1', fontsize = 14)
plt.ylabel('PC-2', fontsize = 14)
plt.show()

In [ ]:
pc1_df = pca_df[(pca_df['x'] > 1) & (pca_df['y'] < 1)].reset_index(drop=True)
print(pc1_df.loc[159]['violation_text'])
print('---')
print(pc1_df.loc[100]['violation_text'])

This is a tSNE of the initial dataset using `learning_rate=100` and `perplexity=40`

In [ ]:
X = df_all['violation_data']
y = df_all['facility_type']
min_df = 0.01
max_df = 0.90
vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))
X_dtm = vect.fit_transform(X)

In [ ]:
start = time.time()

model = TSNE(learning_rate=100, perplexity=40)
transformed = model.fit_transform(X_dtm.toarray())

end = time.time()
print('Elapsed time: ', end - start, 'seconds')

In [ ]:
tsne_df = pd.DataFrame(transformed, columns=['x', 'y'])
tsne_df['violation_text'] = df_all['trim_violation_text']
tsne_df['facility_type'] = df_all['facility_type']
tsne_df['Total_Settlement'] = df_all['Total']
tsne_df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.lmplot( x="x", y="y", data=tsne_df, fit_reg=False, hue='facility_type', legend=False)
plt.legend(loc='lower right')
plt.show()

This is a SVD of the initial dataset using `n_components = 2`

In [ ]:
X = df_all['violation_data']
y = df_all['facility_type']
min_df = 0.01
max_df = 0.90
vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))
X_dtm = vect.fit_transform(X)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2)
svd_ctv = svd.fit_transform(X_dtm)

In [ ]:
svd_df = pd.DataFrame(svd_ctv, columns=['x', 'y'])
svd_df['violation_text'] = df_all['trim_violation_text']
svd_df['facility_type'] = df_all['facility_type']
svd_df['Total_Settlement'] = df_all['Total']
svd_df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.lmplot( x="x", y="y", data=svd_df, fit_reg=False, hue='facility_type', legend=False)
plt.legend(loc='lower right')
plt.show()

## k-Means on the initial dataset

In [ ]:
X = df_all['trim_violation_text']
y = df_all['facility_type']
min_df = 0.01
max_df = 0.90
vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))
X_dtm = vect.fit_transform(X)
print('Size of Feature Vector is: ', len(X_dtm.toarray()[0]))

In [ ]:
start = time.time()

metric_ = []
silhouette_avg_ = []
for i in range(2, 25):
    kmeans = KMeans(n_clusters=i, random_state=10)
    kmeans.fit(X_dtm)
    metric_.append(kmeans.inertia_)
    labels = kmeans.predict(X_dtm)
    silhouette_avg = silhouette_score(X_dtm, labels)
    silhouette_avg_.append(silhouette_avg)

end = time.time()
print('Elapsed time: ', end - start, 'seconds')

In [ ]:
cluster = [x for x in range(2, 25)]
plt.scatter(cluster, metric_)
plt.plot(cluster, metric_)
plt.ylabel('k-means Inertia')
plt.xlabel('Number of Clusters')
plt.title('Elbow Chart of k-Means Clusters')
plt.show()

In [ ]:
cluster = [x for x in range(2, 25)]
plt.scatter(cluster, silhouette_avg_)
plt.plot(cluster, silhouette_avg_)
plt.ylabel('k-means Silhouette Average')
plt.xlabel('Number of Clusters')
plt.title('Silhouette Average of k-Means Clusters')
plt.show()

In [ ]:
silhouette_avg_

In [ ]:
silhouette_avg_[5]

In [ ]:
X_dtm = vect.fit_transform(X)
kmeans = KMeans(n_clusters=7, random_state=10)
kmeans.fit(X_dtm)
labels = kmeans.predict(X_dtm)
labels

In [ ]:
kmeans_df = pd.DataFrame()
kmeans_df['violation_text'] = df_all['trim_violation_text']
kmeans_df['facility_type'] = df_all['facility_type']
kmeans_df['Total_Settlement'] = df_all['Total']
kmeans_df['Cluster'] = labels
kmeans_df['notification'] = df_all['notification']
kmeans_df['facility_id'] = df_all['facility id']
kmeans_df.head(10)

In [ ]:
kmeans_df[kmeans_df['Cluster'] ==  2].to_csv('cluster2.csv', index=False)

In [ ]:
facility_type_ = dict()
type_list = range(0, 7)
for each in type_list:
    text = ' '.join(kmeans_df.loc[kmeans_df['Cluster'] == each, 'violation_text'].values)
    facility_type_[each] = word_tokenize(text)

In [ ]:
Counter(facility_type_[0]).most_common(20)

In [ ]:
Counter(facility_type_[1]).most_common(20)

In [ ]:
Counter(facility_type_[2]).most_common(20)

In [ ]:
Counter(facility_type_[3]).most_common(20)

In [ ]:
Counter(facility_type_[4]).most_common(20)

In [ ]:
Counter(facility_type_[5]).most_common(20)

In [ ]:
Counter(facility_type_[6]).most_common(20)

In [ ]:
group_zero = Counter(facility_type_[0]).most_common(100)
group_one = Counter(facility_type_[1]).most_common(100)
group_two = Counter(facility_type_[2]).most_common(100)
group_three = Counter(facility_type_[3]).most_common(100)
group_four = Counter(facility_type_[4]).most_common(100)
group_five = Counter(facility_type_[5]).most_common(100)
group_six = Counter(facility_type_[6]).most_common(100)

In [ ]:
def create_wordcloud(d):
    wordcloud = WordCloud(background_color='white',
                          max_words=50, 
                          max_font_size=40,
                          random_state=42).generate(str(d))
    return wordcloud

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(30, 30))

ax = axes[0, 0]
ax.imshow(create_wordcloud(group_zero), interpolation="bilinear")
ax.axis('off')
ax.set_title("Cluster 0", fontsize=30)

ax = axes[0, 1]
ax.imshow(create_wordcloud(group_one))
ax.axis('off')
ax.set_title("Cluster 1", fontsize=30)

ax = axes[1, 0]
ax.imshow(create_wordcloud(group_two))
ax.axis('off')
ax.set_title("Cluster 2", fontsize=30)

ax = axes[1, 1]
ax.imshow(create_wordcloud(group_three))
ax.axis('off')
ax.set_title("Cluster 3", fontsize=30)

ax = axes[2, 0]
ax.imshow(create_wordcloud(group_four))
ax.axis('off')
ax.set_title("Cluster 4", fontsize=30)

ax = axes[2, 1]
ax.imshow(create_wordcloud(group_five))
ax.axis('off')
ax.set_title("Cluster 5", fontsize=30)
plt.show()

ax = axes[3, 0]
ax.imshow(create_wordcloud(group_six))
ax.axis('off')
ax.set_title("Cluster 6", fontsize=30)
plt.show()

In [ ]:
df_cross_tab = kmeans_df.iloc[:, 1:4]
cross_tab_summary = pd.crosstab(df_cross_tab['facility_type'], df_cross_tab['Cluster'])
cross_tab_summary 

In [ ]:
start = time.time()

model = TSNE(learning_rate=100, perplexity=50)
transformed = model.fit_transform(X_dtm.toarray())

end = time.time()
print('Elapsed time: ', end - start, 'seconds')

In [ ]:
tsne_df = pd.DataFrame(transformed, columns=['x', 'y'])
tsne_df['violation_text'] = kmeans_df['violation_text']
tsne_df['facility_type'] = kmeans_df['facility_type']
tsne_df['Total_Settlement'] = kmeans_df['Total_Settlement'] 
tsne_df['Cluster'] = labels

In [ ]:
tsne_df.head()

In [ ]:
tsne_df.shape

In [ ]:
tsne_df['violation_text'].isnull().sum()

In [ ]:
tsne_df = tsne_df.dropna(axis=0, how='any')

In [ ]:
tsne_df.shape

In [ ]:
plt.figure(figsize=(20,10))
sns.lmplot( x="x", y="y", data=tsne_df, fit_reg=False, hue='Cluster', legend=False)
plt.legend(loc='lower right')
plt.show()

In [ ]:
def tsne_run(lr, perp):
    start = time.time()

    X = df_all['violation_data']
    y = df_all['facility_type']

    min_df = 0.01
    max_df = 0.90

    vect = CountVectorizer(stop_words='english', min_df=min_df, max_df=max_df, ngram_range=(3,4))
    X_dtm = vect.fit_transform(X)

    kmeans = KMeans(n_clusters=6, random_state=10)
    kmeans.fit(X_dtm)
    labels = kmeans.predict(X_dtm)
    
    kmeans_df = pd.DataFrame()
    kmeans_df['violation_text'] = df_all['trim_violation_text']
    kmeans_df['facility_type'] = df_all['facility_type']
    kmeans_df['Total_Settlement'] = df_all['Total']
    kmeans_df['Cluster'] = labels

    model = TSNE(learning_rate=lr, perplexity=perp)
    transformed = model.fit_transform(X_dtm.toarray())

    tsne_df = pd.DataFrame(transformed, columns=['x', 'y'])
    tsne_df['violation_text'] = kmeans_df['violation_text']
    tsne_df['facility_type'] = kmeans_df['facility_type']
    tsne_df['Total_Settlement'] = kmeans_df['Total_Settlement'] 
    tsne_df['Cluster'] = labels

    plt.figure(figsize=(20,10))
    sns.lmplot( x="x", y="y", data=tsne_df, fit_reg=False, hue='Cluster', legend=False)
    plt.legend(loc='lower right')
    plt.show()

    end = time.time()
    print('Elapsed time: ', end - start, 'seconds')

In [ ]:
tsne_run(90, 50)

In [ ]:
tsne_run(100, 50)

In [ ]:
tsne_run(150, 50)

In [ ]:
tsne_run(200, 50)

In [ ]:
tsne_run(250, 50)

In [ ]:
#https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python
#https://pythonprogramming.net/stemming-nltk-tutorial/

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
lmtzr = WordNetLemmatizer()
ps = PorterStemmer()

In [ ]:
a = 'failure to operate and maintain a continuous monitoring system cms according to rule plan'
a_list = a.split(' ')
list_= [lmtzr.lemmatize(each) for each in a_list]
b = ' '.join(list_)
a_list = b.split(' ')
list_= [ps.stem(each) for each in a_list]
a = ' '.join(list_)

In [ ]:
a

In [ ]:
b